# House-prices-modeling

### 1. Data Setup

In [1]:
import pandas as pd

# dont use absoulte path, use relative one .../ !!!
df_master = pd.read_csv('train.csv')
df = df_master.copy()

In [2]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### 2. Feature Selection

Continuous Features:

- LotFrontage: Linear feet of street connected to property
- LotArea: Lot size in square feet

Categorical Features:

- Neighborhood: Physical locations within Ames city limits
- BldgType: Type of dwelling
- MSZoning: Identifies the general zoning classification of the sale.

In [3]:
df['LotFrontage']

0       65.0
1       80.0
2       68.0
3       60.0
4       84.0
        ... 
1455    62.0
1456    85.0
1457    66.0
1458    68.0
1459    75.0
Name: LotFrontage, Length: 1460, dtype: float64

In [4]:
df['Neighborhood']

0       CollgCr
1       Veenker
2       CollgCr
3       Crawfor
4       NoRidge
         ...   
1455    Gilbert
1456     NWAmes
1457    Crawfor
1458      NAmes
1459    Edwards
Name: Neighborhood, Length: 1460, dtype: object

### 3. Feature processing
#### - Missing values

In [5]:
# First check if there are missing values

columns = ['LotFrontage', 'LotArea', 'Neighborhood', 'BldgType', 'MSZoning']
for column in columns:
    null_rows = df[df[column].isnull()]
    if null_rows.empty:
        print('No missing values in column', column)
    else:
        print('Missing values in column', column, 'in the rows:')
        print(null_rows)

Missing values in column LotFrontage in the rows:
        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
7        8          60       RL          NaN    10382   Pave   NaN      IR1   
12      13          20       RL          NaN    12968   Pave   NaN      IR2   
14      15          20       RL          NaN    10920   Pave   NaN      IR1   
16      17          20       RL          NaN    11241   Pave   NaN      IR1   
24      25          20       RL          NaN     8246   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1429  1430          20       RL          NaN    12546   Pave   NaN      IR1   
1431  1432         120       RL          NaN     4928   Pave   NaN      IR1   
1441  1442         120       RM          NaN     4426   Pave   NaN      Reg   
1443  1444          30       RL          NaN     8854   Pave   NaN      Reg   
1446  1447          20       RL          NaN    26142   Pave   NaN      IR1   

 

So the column 'LotFrontage' has missing values, specifically 259 missing values (size of the matrix). I want to replace those values with the mean so i don't alter so much the distribution of the data. 

In [6]:
# calculating the mean
mean_lotfrontage = df['LotFrontage'].mean()

# replacing the missing values with the mean
df['LotFrontage'].fillna(mean_lotfrontage, inplace=True)

In [7]:
# let's do it again to check that it was done correctly
columns = ['LotFrontage']
for column in columns:
    null_rows = df[df[column].isnull()]
    if null_rows.empty:
        print('No missing values in column', column)
    else:
        print('Missing values in column', column, 'in the rows:')
        print(null_rows)

No missing values in column LotFrontage


#### - Duplicates 

In [8]:
# Check for duplicates 
useful_features = ['Foundation', 'KitchenQual', 'TotRmsAbvGrd', 'WoodDeckSF', 'YrSold', '1stFlrSF']
help(df[useful_features].duplicated)

Help on method duplicated in module pandas.core.frame:

duplicated(subset: 'Hashable | Sequence[Hashable] | None' = None, keep: "Literal['first', 'last', False]" = 'first') -> 'Series' method of pandas.core.frame.DataFrame instance
    Return boolean Series denoting duplicate rows.
    
    Considering certain columns is optional.
    
    Parameters
    ----------
    subset : column label or sequence of labels, optional
        Only consider certain columns for identifying duplicates, by
        default use all of the columns.
    keep : {'first', 'last', False}, default 'first'
        Determines which duplicates (if any) to mark.
    
        - ``first`` : Mark duplicates as ``True`` except for the first occurrence.
        - ``last`` : Mark duplicates as ``True`` except for the last occurrence.
        - False : Mark all duplicates as ``True``.
    
    Returns
    -------
    Series
        Boolean series for each duplicated rows.
    
    See Also
    --------
    Index.duplicat

In [9]:
df[useful_features].duplicated(keep='first').sum()

32

In [10]:
#df = df[~df[useful_features].duplicated(keep='first')]
# or just use drop_duplicates
df.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [11]:
df = df.reset_index(drop=True)
df.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [12]:
label_col = 'SalePrice'
df_tmp = df_master[useful_features+[label_col]]
df_tmp[df_tmp.duplicated(keep=False)]

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,SalePrice
145,PConc,Gd,7,0,2006,970,130000
193,PConc,Gd,7,0,2006,970,130000


In [13]:
df_tmp = df_master[useful_features+[label_col]]
df_tmp[df_tmp[useful_features].duplicated(keep=False)]

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,SalePrice
18,PConc,Gd,6,0,2008,1114,159000
76,CBlock,TA,4,0,2008,952,135750
87,PConc,Gd,4,0,2009,612,164500
89,PConc,TA,5,0,2007,990,123600
102,Slab,TA,8,0,2009,1535,118964
126,CBlock,TA,5,0,2007,958,128000
145,PConc,Gd,7,0,2006,970,130000
193,PConc,Gd,7,0,2006,970,130000
194,CBlock,TA,5,0,2008,864,127000
203,PConc,Gd,3,149,2008,848,149000


In [14]:
df_tmp2 = df_tmp[df_tmp[useful_features].duplicated(keep=False)]
df_tmp2[(df_tmp2['Foundation'] == 'PConc') & (df_tmp2['KitchenQual'] == 'Gd') & (df_tmp2['TotRmsAbvGrd'] == 6)]

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,SalePrice
18,PConc,Gd,6,0,2008,1114,159000
282,PConc,Gd,6,172,2009,1314,207500
511,PConc,Gd,6,132,2006,1368,202665
1157,PConc,Gd,6,172,2009,1314,230000
1306,PConc,Gd,6,132,2006,1368,202500
1341,PConc,Gd,6,0,2008,1114,155000


### Scaling continuous features

In [15]:
continuous_columns = df[useful_features].select_dtypes(include='number').columns
continuous_columns

Index(['TotRmsAbvGrd', 'WoodDeckSF', 'YrSold', '1stFlrSF'], dtype='object')

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df[continuous_columns])

StandardScaler()

In [17]:
scaled_columns = scaler.transform(df[continuous_columns])
scaled_columns

array([[ 0.91220977, -0.75217584,  0.13877749, -0.79343379],
       [-0.31868327,  1.62619479, -0.61443862,  0.25714043],
       [-0.31868327, -0.75217584,  0.13877749, -0.62782603],
       ...,
       [ 1.52765629, -0.75217584,  1.64520971,  0.06565646],
       [-0.93412978,  2.16891024,  1.64520971, -0.21898188],
       [-0.31868327,  5.12192075,  0.13877749,  0.2416147 ]])

In [18]:
continuous_features_df = pd.DataFrame(data=scaled_columns, columns=continuous_columns)
continuous_features_df.head()

,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF
0,0.912210,-0.752176,0.138777,-0.793434
1,-0.318683,1.626195,-0.614439,0.257140
2,-0.318683,-0.752176,0.138777,-0.627826
3,0.296763,-0.752176,-1.367655,-0.521734
4,1.527656,0.780197,0.138777,-0.045611


### Encoding categorical variables
Encoding features refers to the process of converting categorical or textual data into numerical representations that can be used as input for machine learning models

In [19]:
categorical_columns = df[useful_features].select_dtypes(include='object').columns
categorical_columns

Index(['Foundation', 'KitchenQual'], dtype='object')

In [20]:
categorical_features_df = pd.get_dummies(df[categorical_columns])
categorical_features_df.head()

,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA
0,0,0,1,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,1,0


In [21]:
final_df = continuous_features_df.join(categorical_features_df).join(df[label_col])
final_df.head()

,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,SalePrice
0,0.912210,-0.752176,0.138777,-0.793434,0,0,1,0,0,0,0,0,1,0,208500
1,-0.318683,1.626195,-0.614439,0.257140,0,1,0,0,0,0,0,0,0,1,181500
2,-0.318683,-0.752176,0.138777,-0.627826,0,0,1,0,0,0,0,0,1,0,223500
3,0.296763,-0.752176,-1.367655,-0.521734,1,0,0,0,0,0,0,0,1,0,140000
4,1.527656,0.780197,0.138777,-0.045611,0,0,1,0,0,0,0,0,1,0,250000


### 4. Model training
Using Linear Regression

- Split dataset 

In [22]:
X, y = final_df.drop(columns=[label_col]), final_df[label_col]

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

- Train model

In [24]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [25]:
model.fit(X_train, y_train)

LinearRegression()

### 5. Model evaluation 


In [26]:
y_pred = model.predict(X_test)

In [27]:
y_pred[y_pred < 0] = 0

In [28]:
import numpy as np
from sklearn.metrics import mean_squared_log_error

def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

In [29]:
compute_rmsle(y_test, y_pred)

0.22